In [ ]:
import paramiko
import time
import sys

ip = "172.16.16.122"
user = "root"
pw = "asd123!@"

# ---- 원격 실행 및 실시간 출력 함수 ----
def run_remote(script):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(ip, username=user, password=pw, port=22)

    shell = ssh.invoke_shell()
    shell.send(script + "\n")

    while True:
        if shell.recv_ready():
            output = shell.recv(4096).decode('utf-8', errors='ignore')
            print(output, end='')
            sys.stdout.flush()
        # exit_status_ready는 단순 명령에는 바로 True가 되지 않으므로, timeout 기반 루프
        if shell.closed:
            break
        time.sleep(0.1)

    ssh.close()


def lamp_ftp() : ## 완료
    return """
    #!/bin/bash
set -e

echo "===== Rocky Linux 서버 초기화 스크립트 시작 ====="

# 0️⃣ firewalld 비활성화
echo "firewalld 비활성화 중..."
systemctl stop firewalld
systemctl disable firewalld
echo "firewalld 비활성화 완료!"


# 3️⃣ Apache 설치
echo "Apache 설치 중..."
dnf -y install httpd
systemctl enable --now httpd
echo "Apache 설치 및 자동 시작 완료!"

# 4️⃣ 사용자 웹 디렉토리 생성
WEB_DIR="/home/heeyoung"
mkdir -p "$WEB_DIR"
chmod 755 "$WEB_DIR"
chown -R $USER:$USER "$WEB_DIR"

# SELinux 활성화 시 컨텍스트 적용
if sestatus | grep -q "enabled"; then
    chcon -R -t httpd_sys_content_t "$WEB_DIR"
fi
echo "웹 디렉토리 생성 및 권한 설정 완료!"

# 5️⃣ VirtualHost 설정
VHOST_CONF="/etc/httpd/conf.d/heeyoung.conf"
cat <<EOF > "$VHOST_CONF"
<VirtualHost *:80>
    ServerName www.heeyoung.com
    DocumentRoot "$WEB_DIR"
    <Directory "$WEB_DIR">
        AllowOverride None
        Options Indexes FollowSymLinks
        Require all granted
    </Directory>
</VirtualHost>
EOF
echo "VirtualHost 설정 완료: $VHOST_CONF"

# 6️⃣ index.html 생성
echo "qqq" > "$WEB_DIR/index.html"
echo "index.html 생성 완료!"

# 7️⃣ Apache 재시작
systemctl restart httpd
echo "Apache 재시작 완료!"

# 8️⃣ vsftpd 설치 및 시작
echo "vsftpd 설치 중..."
dnf -y install vsftpd
systemctl enable --now vsftpd
echo "vsftpd 설치 및 자동 시작 완료!"

echo "🎉 모든 설정 완료! 웹 서버와 FTP 서버가 준비되었습니다."

    """


bash_script = lamp_ftp()

run_remote(bash_script)